# Toronto Neighborhood Analysis

All of the answers are combined together for easiness of the checker

## Question 1

In [1]:
#!conda install -c conda-forge geopy
!pip install geopy


In [2]:
#!conda install -c conda-forge folium=0.5.0 --yes
!pip install folium

     |████████████████████████████████| 102kB 7.7MB/s ta 0:00:011


In [3]:
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import requests
import folium
import lxml
import json
from geopy.geocoders import Nominatim
from pandas.io.json import json_normalize
from sklearn import cluster
import matplotlib.cm as cm
import matplotlib.colors as colors

#### reading table from the link and removing the not assigned values in 'Borough'

In [7]:
url_1 = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
page_1 = requests.get(url_1)
soup_1 = BeautifulSoup(page_1.content, 'html.parser')
Table_1 = soup_1.find("table",{"class":"wikitable sortable"})

Temp_df_1 = pd.read_html(str(Table_1))[0]

Temp_df_2 = Temp_df_1[Temp_df_1.Borough != 'Not assigned']

Temp_df_2.reset_index(inplace = True)
Temp_df_2

,index,Postal Code,Borough,Neighborhood
0,2,M3A,North York,Parkwoods
1,3,M4A,North York,Victoria Village
2,4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,5,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
5,8,M9A,Etobicoke,Islington Avenue
6,9,M1B,Scarborough,"Malvern, Rouge"
7,11,M3B,North York,Don Mills
8,12,M4B,East York,"Parkview Hill, Woodbine Gardens"
9,13,M5B,Downtown Toronto,"Garden District, Ryerson"


#### If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough.

In [8]:
Temp_df_2['Neighborhood'] = np.where(Temp_df_2['Neighborhood'] == 'Not assigned',Temp_df_2['Borough'], Temp_df_2['Neighborhood'])

/opt/conda/envs/Python36/lib/python3.6/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


#### Combining the addresses with same postal code

In [9]:
df = Temp_df_2.groupby(['Postal Code','Borough'], sort=False).agg(', '.join)

df = df.reset_index()
df

,Postal Code,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
5,M9A,Etobicoke,Islington Avenue
6,M1B,Scarborough,"Malvern, Rouge"
7,M3B,North York,Don Mills
8,M4B,East York,"Parkview Hill, Woodbine Gardens"
9,M5B,Downtown Toronto,"Garden District, Ryerson"


#### printing the shape

In [10]:
df.shape

(103, 3)